### TP 4 EX 1
Grupo 5:
     
     Breno Fernando Guerra Marrão A97768
     
     Tales André Rovaris Machado A96314

## Inicialização
Usamos a biblioteca pysmt para resolver o problema proposto 

In [3]:
from pysmt.shortcuts import *
from pysmt.typing import *


## Inicialização de variáveis

In [89]:
START = Int(0)
FREE = Int(1) # No modo Free  não existe qualquer força de travagem
STOPPING = Int(2)
BLOCKED = Int(3) # no modo Blocked as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa)
STOPPED = Int(4) # no modo Stopped o veículo está imobilizado.

a = 0.02 # Forca de atrito
P = 1000 #peso
Vi = 20 # Velocidade inicial
T = 0.2 # Segundos maximo q pode ficar tanto no free como no blockd



Declararação das variáveis do FOTS correspondente ao veículo em que t é o tempo , m o estado que o veículo se encontra , V a velocidade do corpo em relação ao solo, V velocidade linear das rodas em relação ao solo.

In [2]:
def declare(i):
    s = {}
    s['t'] = Symbol('t'+str(i),REAL)
    s['m'] = Symbol('m'+str(i),INT)
    s['V'] = Symbol('V'+str(i),REAL)
    s['v'] = Symbol('v'+str(i),REAL)
    s['cr'] = Symbol('cronometro'+str(i),REAL)

    return s

O estado inicial do FOTS é o seguinte :
$$
m = \mathsf{START}\wedge v = V_{0}  \wedge V_{0} = V \wedge t = 0 \wedge V_{0}  >= 0 \wedge cr = 0 \\
$$
Temos que começar no Start e definimos V e v com o valor de V0 que é o "input" do problema , e que diz a velocidade inicial do veículo, e cr que vai marcar a diferença de tempo .

In [69]:
def init(s):
    return And(Equals(s['m'],START),Equals(s['V'],Real(Vi)+0.1),s['V'] >= 0.0,Equals(Real(Vi),s['v']),Equals(s['t'], Real(0)),Equals(s['cr'], Real(0)))


As transiçõe "untimed" do fots são as seguintes : 
$$
\begin{array}{c}
m = \mathsf{START} \wedge m' = \mathsf{FREE} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c' = c \\
\vee\\
m = \mathsf{FREE} \wedge m' = \mathsf{STOPPING} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c = T \wedge c' = 0  \\
\vee\\
m = \mathsf{FREE} \wedge m' = \mathsf{STOPPING} \wedge v' = v \wedge V' = V \wedge t' = t \wedge V = 0 \wedge c' = 0  \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{STOPPED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \wedge V = 0 \wedge c' = 0 \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{BLOCKED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \wedge V >= 0 \wedge c' = 0 \\
\vee\\
m = \mathsf{BLOCKED} \wedge m' = \mathsf{FREE} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c' = 0 \wedge c = T \\
\vee\\
\end{array}
\\
$$

Iremos aproximar $\dot{V}$ por $\frac{V'-V}{t'-t}$ e $\dot{v}$ por $\frac{v'-v}{t'-t}$

As transições "timed " -  switches do fots são as seguintes :
Para as transições freeFree e stoppingStopping :   $\dot{V}$ =  - F obtemos $\frac{V'-V}{t'-t} = - F $ e agora multiplicando nos dois lados (t'-t) obtemos $V'-V = (-F) * (t' - t)$ que é a mesma coisa que dizer $V' - V = (-c *(V-v)) * (t'-t)$ aplicando este mesmo processo para $\dot{v} = - a * P + F$ obtemos $v' - v = (-a * P + c * (V-v)) * (t'-t)\newline$
Para a transição blockedBlocked $\dot{v} = -a * P$ é o mesmo que dizer $v' - v = (- a * P ) * (t'-t)$
$$
\begin{array}{c}
(m = \mathsf{FREE} \wedge m' = \mathsf{FREE} \wedge v' < v \wedge t' > t \wedge V' < V \wedge v' \geq 0 \wedge V' \geq 0 \wedge cr = 0 \wedge t' - t = T \wedge cr' = t' - t \wedge  V' - V = (-c *(V-v)) * (t'-t) \wedge  v' - v = (-a * P + c * (V-v)) * (t'-t)) \\
\vee\\
(m = \mathsf{STOPPING} \wedge m' = \mathsf{STOPPING} \wedge v' < v \wedge t' > t  \wedge V' < V \wedge v' \geq 0 \wedge V' \geq 0 \wedge cr = 0  \wedge cr' = 0 \wedge  V'-V = (-F) * (t' - t) \wedge  v' - v = (-a * P + c * (V-v)) * (t'-t) \wedge v >= 0 )\\
\vee\\
(m = \mathsf{BLOCKED} \wedge m' = \mathsf{BLOCKED} \wedge v' < v \wedge t' > t  \wedge V' < V \wedge v' \geq 0 \wedge V' \geq 0 \wedge cr = 0 \wedge t' - t >= T \wedge cr' = t' - t \wedge  V'-V = (-F) * (t' - t) \wedge  v' - v = (-a * P + c * (V-v)) * (t'-t)) \\
\end{array}
\\
$$

In [124]:
def trans(s,p):
    #Untimed
    #Falta colocar as alterações de velocidade
    #Definir condições para trocar entre as variaveis que vai no timed ................
    #Stopping momento até bloquear definir
    #CR talvez não colocar condição ver
    startFree = And(Equals(s['m'],START),Equals(p['m'],FREE),s['t'] >= 0,p['t'] >= 0,Equals(s['v'],p['v']) ,Equals(s['V'],p['V']),Equals(s['cr'],p['cr'])) 
    freeStopping = And(Equals(s['m'],FREE),Equals(p['m'],STOPPING),Equals(s['v'],p['v']),Equals(p['cr'],Real(0)),Equals(s['V'],p['V'])
                       ,Equals(p['t'],s['t']),Equals(s['cr'], Real(T)),s['t'] >= 0,p['t'] >= 0)
    freeStopping1 = And(Equals(s['m'],FREE),Equals(p['m'],STOPPING),Equals(s['v'],p['v']),Equals(p['cr'],Real(0)),
                        Equals(s['V'],Real(0)),Equals(s['V'],p['V']),Equals(p['t'],s['t']))
    stoppingBlocked = And(Equals(s['m'],STOPPING),Equals(p['m'],BLOCKED), Equals(s['v'],p['v']),
                          Equals(s['v'],Real(0)) ,Equals(s['V'],p['V']))
    '''
    #stoppingStopped = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPED), Equals(s['v'],p['v']) ,Equals(s['V'],p['V'])
                          ,Equals(s['V'],Real(0)), Equals(s['v'],Real(0)),GT(s['V'],Real(0)),Equals(p['cr'],Real(0)))
    #blockedFree =  And(Equals(s['m'],BLOCKED),Equals(p['m'],FREE), Equals(s['v'],p['v']) ,Equals(p['cr'],Real(0)),Equals(s['V'],p['V']),Equals(s['cr'],Real(0)))
    #blockedStopping =  And(Equals(s['m'],BLOCKED),Equals(p['m'],Stopping), Equals(s['v'],p['v']) Equals(p['cr'],Real(0)),
                       Equals(s['V'],p['V']),(Equals(s['V'],Real(0))), (Equals(s['v'],Real(0))))
    '''
    
    #DAR PRO FreeFree uma condição pra ele parar de fazer freeFree sem ser a do timer 
    #provavel colocar alguma porcentagem de diferenca de vel q nem no termostato 
    #Timed - Switchs
    freeFree = And(Equals(s['m'],FREE), Equals(p['m'],FREE),s['t'] < p['t'],s['V'] > p['V'],s['v'] > p['v'],
                  p['v'] >= 0 , p['V']>= 0 , Equals(Real(T), p['t'] -s['t'] ), Equals(s['cr'], Real(0)),
                  Equals(p['cr'],Real(T)),cond(s['V'],s['v'],s['t'],p['V'],p['v'],p['t'],Real(0.1)))
    blockedBlocked = And(Equals(s['m'],BLOCKED),Equals(p['m'],BLOCKED), Equals(Real(T), p['t'] -s['t']),
                         s['t'] < p['t'],s['V'] > p['V'],p['v'] >= 0 , p['V']>= 0, Equals(p['V'] ,p['v']),
                        Equals(p['cr'],Real(T)),Equals(p['v']-s['v'],(-a*P ) * (p['t'] - s['t']))) 
    stoppingStopping = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPING),s['V'] > p['V'],p['v'] >= 0 , p['V']>= 0 , s['v'] > p['v'],
                 s['t'] < p['t'],cond(s['V'],s['v'],s['t'],p['V'],p['v'],p['t'],Real(2)))
    #   return Or(startFree ,freeStopping,stoppingBlocked ,blockedFree ,stoppingStopped,freeFree,blockedBlocked,stoppingStopping)
    return Or(startFree ,freeStopping,freeStopping1,freeFree,stoppingStopping,stoppingBlocked,blockedBlocked)
def cond (V0,v0,t0,V,v,t,c):
    return And(Equals(V-V0,(-c * (V0-v0)) * (t -t0)),Equals(v-v0,(-a*P + (c * (V0-v0))) * (t - t0)))



In [126]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        # completar
        trace = [declare(i) for i in range(k)]
        
        #semantics
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", (s.get_value(trace[i][v])))
                print("------------------------")
    
                        
####
gera_traco(declare,init,trans,6)

SolverReturnedUnknownResultError: 

## 1. ”o veículo imobiliza-se completamente em menos de t segundos” 
Que pode ser traduzido por 
sendo T o número de segundos que queremos limitar sendo S o conjunto dos estados .

$$
\begin{array}{c}

(s_{t} <= T \wedge s_{v} = 0 \wedge s_{V} = 0) \vee (s_{t} > T)
\end{array}

$$


In [23]:
def termina_t(s,tempo):
    return Or(And(GE(s['t'],Int(tempo)),Equals(s['v'],Int(0)),Equals(s['V'],Int(0))),Not(GE(s['t'],Int(tempo))))

def bmc_always1(declare,init,trans,inv,t,K):
    
    
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            trace = [declare(i) for i in range(k)]
        
            s.add_assertion(init(trace[0]))
        
            for i in range(k-1):
                s.add_assertion(trans(trace[i],trace[i+1]))
            
            s.add_assertion(Not(And(inv(trace[i],t) for i in range(k-1))))        
            if s.solve():
                for i in range(k):
                    print("Passo", i)
                    for v in trace[i]:
                        print(v,"=",s.get_value(trace[i][v]))
                    print("----------------")
                print("Tem erro")
                return
    print("Erro não encontrado")

    
print(bmc_always1(declare,init,trans,termina_t,5,10))

(((5 <= t0) & (v0 = 0) & (V0 = 0)) | (! (5 <= t0)))


## 2.“a velocidade V diminui sempre com o tempo”
Que pode ser traduzido por
Sendo S o conjunto dos estados .
Para qualquer i < length(s) : se s_{i}{t} < s_{i+1}{t} então s_{i}{V} > s_{i+1}{V}

$$
\forall_{i\in length(s)}  \quad  se  \quad  s_{i,t} < s_{i+1,t}\quad então \quad s_{i,V} > s_{i+1,V}
$$

In [16]:
p = declare(1)
def veldiminui(c,p):
    return Or(And(GT(c['t'],p['t']),GT(p['V'],c['V'])), Not(GT(c['t'],p['t'])))
def bmc_always2(declare,init,trans,inv,K):
    
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            trace = [declare(i) for i in range(k)]
        
            s.add_assertion(init(trace[0]))
        
            for i in range(k-1):
                s.add_assertion(trans(trace[i],trace[i+1]))
                s.add_assertion(Not(And(inv(trace[i],trace[i+1]))))        

            if s.solve():
                for i in range(k):
                    print("Passo", i)
                    for v in trace[i]:
                        print(v,"=",s.get_value(trace[i][v]))
                    print("----------------")
                print("Tem erro")
                return
    print("Erro não encontrado")
bmc_always2(declare,init,trans,veldiminui,10)     

(((t1 < t0) & (V0 < V1)) | (! (t1 < t0)))
